# Ejercicio 1

#### 1. Defina un conjunto de reglas para el comportamiento del enemigo, como patrones de movimiento y decisiones de ataque
#### y
#### 4. Ajuste las reglas y los parámetros para explorar diferentes estrategias y resultados de los jugadores.

1. El enemigo puede moverse en cuatro direcciones: arriba, abajo, izquierda y derecha.
2. Cada turno, el enemigo decide si moverse o atacar. Si el jugador está cerca (a una distancia de 1 unidad), el enemigo tiene una probabilidad del 70% de atacar y un 30% de moverse en alguna dirección aleatoria. Si el jugador está lejos, tiene un 90% de probabilidad de moverse y un 10% de probabilidad de atacar en la dirección del jugador.
3. Si el enemigo ataca y acierta, el jugador pierde

In [17]:
import random

def movimiento_enemigo(posicion, cerca, escenario):
    if escenario == "agresivo":
        prob_cerca = [0.9, 0.1]  # 90% atacar, 10% mover
        prob_lejos = [0.2, 0.8]  # 20% atacar, 80% mover
    elif escenario == "cauteloso":
        prob_cerca = [0.3, 0.7]  # 30% atacar, 70% mover
        prob_lejos = [0.05, 0.95]  # 5% atacar, 95% mover
    else:  # mixto
        prob_cerca = [0.6, 0.4]  # 60% atacar, 40% mover
        prob_lejos = [0.1, 0.9]  # 10% atacar, 90% mover

    if cerca:
        accion = random.choices(['atacar', 'mover'], prob_cerca)[0]
    else:
        accion = random.choices(['atacar', 'mover'], prob_lejos)[0]

    if accion == 'mover':
        movimiento = random.choice(['arriba', 'abajo', 'izquierda', 'derecha'])
        if movimiento == 'arriba':
            posicion[1] += 1
        elif movimiento == 'abajo':
            posicion[1] -= 1
        elif movimiento == 'izquierda':
            posicion[0] -= 1
        elif movimiento == 'derecha':
            posicion[0] += 1

    return accion, posicion

#### Implemente una simulación de Monte Carlo que genere posibles resultados del comportamiento del enemigo durante una serie de turnos.

In [18]:
def simulacion(turnos=10, escenario="mixto"):
    posicion_jugador = [5, 5]
    posicion_enemigo = [random.randint(0, 10), random.randint(0, 10)]
    for _ in range(turnos):
        distancia = abs(posicion_jugador[0] - posicion_enemigo[0]) + abs(posicion_jugador[1] - posicion_enemigo[1])
        accion, posicion_enemigo = movimiento_enemigo(posicion_enemigo, distancia <= 1, escenario)
        if accion == 'atacar' and distancia <= 1:
            return "fracaso"
    return "exito"

#### Analice la distribución de los posibles resultados y determine la probabilidad de éxito o fracaso del jugador.

In [19]:
escenarios = ["agresivo", "cauteloso", "mixto"]

for esc in escenarios:
    resultados = [simulacion(escenario=esc) for _ in range(1000)]
    prob_exito = resultados.count("exito") / 1000
    prob_fracaso = resultados.count("fracaso") / 1000
    print(f"--- {esc.capitalize()} ---")
    print(f"Probabilidad de éxito: {prob_exito}")
    print(f"Probabilidad de fracaso: {prob_fracaso}")
    print()

--- Agresivo ---
Probabilidad de éxito: 0.851
Probabilidad de fracaso: 0.149

--- Cauteloso ---
Probabilidad de éxito: 0.916
Probabilidad de fracaso: 0.084

--- Mixto ---
Probabilidad de éxito: 0.896
Probabilidad de fracaso: 0.104



Como se puede observar, definimos 3 escenarios donde cambiamos las variables y las probabilidades para entender los cambios en el comportamiento del resultado. Quizas lo mas interesante que podemos destacar es que intentamos aplicar un escenario real a cada cambio. Agresivo, cauteloso y mixto.

# Ejercicio 2

#### 1. Implemente la fórmula de Black-Scholes para calcular el precio teórico de la opción

In [20]:
import numpy as np
import scipy.stats as stats

# 1. Black-Scholes
def precio_call_black_scholes(S, K, r, t, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)
    
    return S * stats.norm.cdf(d1) - K * np.exp(-r * t) * stats.norm.cdf(d2)

#### 2. Simule las trayectorias del precio de las acciones mediante el movimiento browniano geométrico

In [21]:
# 2. Simulación de trayectorias usando MBG (Movimiento Browniano Geométrico)
def simular_mbg(S, K, r, t, sigma, n_trayectorias):
    dt = 1/365  # delta t
    trayectorias = np.zeros((n_trayectorias, int(t/dt)))
    trayectorias[:, 0] = S

    for i in range(1, trayectorias.shape[1]):
        rand_std_norm = np.random.standard_normal(trayectorias.shape[0])
        trayectorias[:, i] = trayectorias[:, i-1] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*rand_std_norm)
    
    return trayectorias

#### Calcule el pago de cada camino y promedielos para estimar el precio de la opción.

In [22]:
# 3. Estimación del precio de la opción con Monte Carlo
def precio_call_monte_carlo(S, K, r, t, sigma, n_trayectorias=10000):
    trayectorias = simular_mbg(S, K, r, t, sigma, n_trayectorias)
    precios_finales = trayectorias[:, -1]
    pagos = np.maximum(precios_finales - K, 0)
    return np.exp(-r*t) * np.mean(pagos)

#### Pruebas

In [23]:
# Parámetros
S = 100  # Precio actual
K = 100  # Precio de ejercicio
r = 0.05  # Tasa de interés libre de riesgo
t = 1  # 1 año hasta el vencimiento
sigma = 0.2  # Volatilidad

precio_bs = precio_call_black_scholes(S, K, r, t, sigma)
precio_mc = precio_call_monte_carlo(S, K, r, t, sigma)

print(f"Precio Call Black-Scholes: {precio_bs:.2f}")
print(f"Precio Call Monte Carlo: {precio_mc:.2f}")

Precio Call Black-Scholes: 10.45
Precio Call Monte Carlo: 10.41


Como se puede observar el comportamiento es el esperado pues el valor entre Black-Scholes y Monte carlo es muy similar con una diferencia muy baja.